In [2]:
import os
import sys

module_path = os.path.abspath(os.path.join('../srdt/api/'))
if module_path not in sys.path:
    sys.path.append(module_path)


import pandas as pd
from srdt_analysis.collections import AlbertCollectionHandler
from srdt_analysis.legi_data import get_legi_data

%load_ext dotenv
#%reload_ext dotenv
%dotenv

In [13]:
# read questions
questions = pd.read_csv("/Users/remi/Downloads/questions_v2.csv", index_col="id")

# custom hack
questions.loc[9]['question'] = questions.loc[9]['question'].lower().replace(" ij ", " indemnité journalière ")
questions.loc[16]['question'] = questions.loc[16]['question'].lower().replace(" cp ", " congés payés ")

/var/folders/y9/zlln1xg565d8w1d8btf1c1b00000gn/T/ipykernel_31364/3675195285.py:5: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  questions.loc[9]['question'] = questions.loc[9]['question'].lower().replace(" ij ", " indemnité journalière ")
/v

In [14]:
# read labels
labels = pd.read_csv("/Users/remi/Downloads/search_eval2.csv")
labels = labels[labels['valid'] == 1]
labels

,question,link,positive,valid,source
0,6,https://code.travail.gouv.fr/fiche-service-pub...,T,1,NaN
2,6,https://code.travail.gouv.fr/fiche-ministere-t...,T,1,NaN
4,6,https://code.travail.gouv.fr/fiche-ministere-t...,T,1,NaN
5,6,https://code.travail.gouv.fr/fiche-ministere-t...,T,1,NaN
6,8,https://code.travail.gouv.fr/fiche-service-pub...,T,1,NaN
7,8,https://code.travail.gouv.fr/fiche-service-pub...,T,1,NaN
8,8,https://code.travail.gouv.fr/fiche-ministere-t...,T,1,NaN
12,8,https://code.travail.gouv.fr/fiche-service-pub...,T,1,NaN
13,8,https://code.travail.gouv.fr/fiche-service-pub...,T,1,NaN
14,8,https://code.travail.gouv.fr/fiche-service-pub...,T,1,NaN


In [15]:
collections = AlbertCollectionHandler()

In [3]:
# load data
docs = pd.read_parquet("/Users/remi/tmp/docs.parquet")
chunks = pd.read_parquet("/Users/remi/tmp/chunks.parquet")

# code 
code = get_legi_data()

In [27]:
all_paths = docs['url'].to_list()

In [25]:

user_question = "Bonjour,  J’ai effectuée un remplacement en CDD dans une micro crèche, mon contrat étant fini depuis le 22 septembre 2023 je suis toujours en attente de mon salaire. Après plusieurs relance auprès de la directrice aucun versement n’a été fait. J’aimerais savoir si elle est en droit de me faire patienter comme cela ou sinon qu’elle sont les délais pour qu’elle puisse me verser mon salaire. Cordialement"

contributions_col = 733
info_col = 734
fiche_sp_col = 738
fiche_mt_col = 735

res_search = collections.search(user_question, [contributions_col, info_col, fiche_mt_col, fiche_sp_col], 64)

len(res_search)

64

In [35]:
def to_row(chunk, in_results: bool, in_positives: bool):
    link = chunk['metadata']['url']
    content = chunk['content']
    score = f'{chunk['score']:.2f}'  
    in_dataset = 1 if link in all_paths else 0
    original_id = docs[docs['url'] == link]['cdtn_id'] if in_dataset == 1 else 'n/a'
    
    return {"element": link, "in_results": in_results, "in_positives": in_positives, "in_dataset": in_dataset, "score": score, "content" : content, "url": link, "original_id": original_id}

async def analyze_question(id: int):
    content = questions.loc[id]['question']
    print(content)

    positives = labels[(labels["question"] == id) & (labels["positive"] == 'T') ]['link']

    
    #res = search_bm25(content)
    
    #res = collections.search(content, [contributions_col, info_col, fiche_mt_col, fiche_sp_col], 128)

    res = combined_search(content)    

    # merge chunks (warning it could be too long!)
    chunks = {}
    for r in res:
        chunk = r['chunk']
        chunk['score'] = r['score']
        # temp fix
        chunk['metadata']['url'] = chunk['metadata']['url'].replace('contributions/', 'contribution/')
        
        url = chunk['metadata']['url']
        elem = url
        if (elem not in chunks):
            chunks[elem] = chunk
        else :
            #print(elem)
            chunks[elem]['content'] = "\n\n".join([chunks[elem]['content'], chunk['content']])
            
    search_res = chunks.values()
    
    #print(search_res)
    

    
    emptyness = pd.Series(None, dtype=int)
    results_df = pd.DataFrame(to_row(sr, 1, 0) for sr in search_res) if len(search_res) > 0 else  pd.DataFrame({"url":[], "element":[], "in_results": emptyness, "in_positives":emptyness})

    missing=[]
    
    for p in positives:
        index = [results_df.index[results_df['url'] == p]][0]
        if len(index):
            results_df.at[index[0], 'in_positives'] = 1
        else:
            missing.append(p)


    inputs = [c['content'][:8192] for c in search_res][:64]
    reranked = collections.rerank(content, inputs)
    
    rr_indices = pd.Series({r["index"]: i for i,r in enumerate(reranked)})

    results_df["rr_score"] = pd.Series({r["index"]: r["score"] for i, r in enumerate(reranked)})

    results_df["reranked"] = rr_indices
    
    results_df = pd.concat([results_df, pd.DataFrame([to_row({"content":docs[docs['url'] == m]['content_chunked'] if m in all_paths else "n/a", "score":0, "metadata":{"url":m}}, 0,1) for m in missing])], ignore_index=True)  

    results_df.sort_values(by='reranked', inplace=True)
    
    return results_df


In [28]:
with pd.option_context('display.width', 500, 'display.max_rows', 200, 'display.max_colwidth', 200):
    # 6 8 11 14 15
    response = await analyze_question(14)
    display(response)

Quelle est la procdure suivre pour demander une dérogation pour que mes techniciens puissent ponctuellement travailler 60 heures par semaine pour un chantier particulier? 

Cest dire rajouter la journée du samedi au reste de la semaine.


,element,in_results,in_positives,in_dataset,score,content,url,original_id,rr_score,reranked
13,https://code.travail.gouv.fr/fiche-ministere-travail/la-duree-legale-du-travail,1,1,1,0.02,"En matière de durée du travail, la réglementation définit précisément les règles applicables et les possibilités d'adaptation ouvertes par la négociation collective. \nCette matière est ...",https://code.travail.gouv.fr/fiche-ministere-travail/la-duree-legale-du-travail,"1380 72a440e404 Name: cdtn_id, dtype: object",0.981131,0.0
10,https://code.travail.gouv.fr/fiche-ministere-travail/le-temps-de-travail-des-jeunes-de-moins-de-18-ans,1,0,1,0.02,"À savoir ! La durée du travail du mineur de 14 à moins de 16 ans travaillant pendant ses vacances scolaires ne peut excéder 35 heures par semaine, ni 7 heures par jour. \nLa durée du travail ...",https://code.travail.gouv.fr/fiche-ministere-travail/le-temps-de-travail-des-jeunes-de-moins-de-18-ans,"1264 e3c27df0bb Name: cdtn_id, dtype: object",0.858006,1.0
8,https://code.travail.gouv.fr/fiche-service-public/duree-du-travail-dun-jeune-de-moins-de-18-ans,1,0,1,0.03,Quelle est la durée de temps de travail autorisée d'un salarié de moins de 18 ans ? Le salarié bénéficie-t-il de dispositions spécifiques ? Nous vous présentons la réglementation en vigue...,https://code.travail.gouv.fr/fiche-service-public/duree-du-travail-dun-jeune-de-moins-de-18-ans,"13426 6d7a8c9838 Name: cdtn_id, dtype: object",0.801315,2.0
3,https://code.travail.gouv.fr/fiche-service-public/duree-du-travail-dun-salarie-a-temps-plein,1,1,1,0.03,"Combien d'heures de travail pouvez-vous effectuer par jour, par semaine, par mois et par an ? Est-il possible de travailler plus longtemps que la durée légale le prévoit ? Nous vous expliquons ...",https://code.travail.gouv.fr/fiche-service-public/duree-du-travail-dun-salarie-a-temps-plein,"13498 abdd8263ba Name: cdtn_id, dtype: object",0.775605,3.0
27,https://code.travail.gouv.fr/fiche-ministere-travail/le-travail-de-nuit,1,0,1,0.02,"Selon Santé publique France, le travail de nuit pris en globalité en considérant le travail de nuit habituel et occasionnel, concerne 4,3 millions de personnes en France. \nMais quelle est pré...",https://code.travail.gouv.fr/fiche-ministere-travail/le-travail-de-nuit,"1413 be4ca37b56 Name: cdtn_id, dtype: object",0.767341,4.0
1,https://code.travail.gouv.fr/fiche-service-public/temps-partiel-dun-salarie-dans-le-secteur-prive,1,0,1,0.03,Quelle est la durée d'un travail à temps partiel ? Un salarié est à temps partiel lorsqu'il travaille pendant une durée inférieure à celle d'un salarié à temps plein. La durée minimale d...,https://code.travail.gouv.fr/fiche-service-public/temps-partiel-dun-salarie-dans-le-secteur-prive,"13409 51afbbbe4e Name: cdtn_id, dtype: object",0.750004,5.0
12,https://code.travail.gouv.fr/fiche-ministere-travail/le-travail-a-temps-partiel-contrat-et-statut-du-salarie,1,0,1,0.02,À savoir ! \nUne exonération de cotisations salariales s'applique au titre des heures complémentaires (et supplémentaires pour les salariés à temps plein) effectuées par les salariés (plus...,https://code.travail.gouv.fr/fiche-ministere-travail/le-travail-a-temps-partiel-contrat-et-statut-du-salarie,"1356 30fbfc0fc5 Name: cdtn_id, dtype: object",0.702938,6.0
21,https://code.travail.gouv.fr/fiche-service-public/travail-le-dimanche-dun-salarie-du-secteur-prive,1,0,1,0.02,"Un salarié peut-il être contraint de travailler le dimanche ? Le dimanche est, en principe, une journée légale de repos. Mais des exceptions existent. On parle alors de dérogation au repos do...",https://code.travail.gouv.fr/fiche-service-public/travail-le-dimanche-dun-salarie-du-secteur-prive,"13143 1e9dda80aa Name: cdtn_id, dtype: object",0.621885,7.0
2,https://code.travail.gouv.fr/fiche-service-public/salarie-a-temps-partiel-quelle-est-la-duree-hebdomadaire-minimale-de-travail,1,0,1,0.03,Votre contrat de travail à temps partiel a nécessairement une d

In [81]:
#1 run evaluation with current search (BGE M3) - deal with article + test rerank et full content

#2 run evaluation with BM25 / Hybrid search 

#3 bonus : run BGE M3 with dense / sparse / ColBert

In [40]:
results = {}
reports = []

max_gen = 13

async def run_search_analysis(i) :
    print(f"Question {i}")
    df = await analyze_question(i)
    
    #with pd.option_context('display.width', 500, 'display.max_rows', 150, 'display.max_colwidth', 200):
    #    display(df[["element", "in_results", "in_positives", "in_negatives", "reranked"]])
    TP = (df['in_results'] & df['in_positives']).sum()
    P = df['in_positives'].sum()
    #print(f"found : {TP} / {P}")

    # print last in search (to adapt top_k and score)
    last_idx_positive = df[(df['in_positives'] == 1) & (df['in_results'] == 1)].index.max()
    last_idx_positive_rr = df[(df['in_positives'] == 1) & (df['in_results'] == 1)]["reranked"].max()

    TP_rr = df[(df["reranked"] < max_gen) & (df["in_positives"] == 1)].shape[0]
    
    report = {
        "index": i, 
        "question": questions.loc[i]['question'],
        "n_positive": P,
        "last_idx_positive": last_idx_positive,
        "true_positives": TP,
        
        "recall": f"{TP/P:.3f}",
        "last_idx_positive_rr": last_idx_positive_rr,
    
        f"true_positives_rr_{max_gen}": TP_rr,
        f"recall_rr_{max_gen}": f"{TP_rr/P:.3f}",        
    }

    return report, df

for i in [6,8,11,14,15]:
    report, result = await run_search_analysis(i)
    results[i] = result;
    reports.append(report);

reports_df = pd.DataFrame(reports)
reports_df.set_index("index", inplace=True)

Question 6
Bonjour,  J’ai effectuée un remplacement en CDD dans une micro crèche, mon contrat étant fini depuis le 22 septembre 2023 je suis toujours en attente de mon salaire. Après plusieurs relance auprès de la directrice aucun versement n’a été fait. J’aimerais savoir si elle est en droit de me faire patienter comme cela ou sinon qu’elle sont les délais pour qu’elle puisse me verser mon salaire. Cordialement,
Question 8
Bonjour,
Actuellement membre du CSE, de la CSSCT et RPX, ma direction souhaite me
changer de roulement à compter de janvier.
Lors de notre première rencontre non officielle, il a été dit que mes
absences mettaient mes collègues en souffrance dû au grand nombre de
remplaçants et que sur l'équipe inverse elle doit séparer un binôme.
Lors d'une seconde rencontre non officielle également, elle me dit qu'il
n'ont rien à reprocher à mon travail seulement que sur les 2 équipes ils
faut redynamiser un peu et séparer toujours le binôme.
Je travaille en roulement amplitude de

In [218]:
# report standard
reports_df

,question,n_positive,last_idx_positive,true_positives,recall,last_idx_positive_rr,true_positives_rr_10,recall_rr_10
index,,,,,,,,
6,"Bonjour, J’ai effectuée un remplacement en CD...",4,65,4,1.000,30.0,1,0.250
8,"Bonjour,\nActuellement membre du CSE, de la CS...",14,62,14,1.000,59.0,8,0.571
11,Bonjour je suis actuellement en conflit avec m...,4,58,4,1.000,19.0,3,0.750
14,Quelle est la procdure suivre pour demander un...,2,29,2,1.000,3.0,2,1.000
15,"Bonjour, Mon employeur m'a employée en CDI mai...",5,27,5,1.000,15.0,4,0.800


In [34]:
# report MB25 only
reports_df

,question,n_positive,last_idx_positive,true_positives,recall,last_idx_positive_rr,true_positives_rr_10,recall_rr_10
index,,,,,,,,
6,"Bonjour, J’ai effectuée un remplacement en CD...",4,71,4,1.000,12.0,2,0.500
8,"Bonjour,\nActuellement membre du CSE, de la CS...",14,103,8,0.571,31.0,3,0.214
11,Bonjour je suis actuellement en conflit avec m...,4,112,3,0.750,4.0,1,0.250
14,Quelle est la procdure suivre pour demander un...,2,69,2,1.000,2.0,1,0.500
15,"Bonjour, Mon employeur m'a employée en CDI mai...",5,75,5,1.000,8.0,4,0.800


In [41]:
# report both
reports_df

,question,n_positive,last_idx_positive,true_positives,recall,last_idx_positive_rr,true_positives_rr_13,recall_rr_13
index,,,,,,,,
6,"Bonjour, J’ai effectuée un remplacement en CD...",4,33,4,1.000,26.0,1,0.250
8,"Bonjour,\nActuellement membre du CSE, de la CS...",14,111,14,1.000,40.0,8,0.571
11,Bonjour je suis actuellement en conflit avec m...,4,90,4,1.000,6.0,3,0.750
14,Quelle est la procdure suivre pour demander un...,2,13,2,1.000,3.0,2,1.000
15,"Bonjour, Mon employeur m'a employée en CDI mai...",5,9,5,1.000,12.0,5,1.000


In [19]:
from retriv import SearchEngine
import spacy
nlp_fr = spacy.load('fr_core_news_md')

docs = pd.read_parquet("/Users/remi/tmp/docs_prepro.parquet")
# remove CCs
docs = docs[docs['idcc'].isnull() | (docs['idcc'] == '0000')]

collection = docs[["cdtn_id", "prepro"]].rename(columns={'cdtn_id': 'id', 'prepro': 'text'}).to_dict(orient='records')

se = SearchEngine("new-index", stemmer=None, stopwords=None, min_df=2,).index(collection)


/Users/remi/.pyenv/versions/3.12.10/envs/play/lib/python3.12/site-packages/spacy/util.py:910: UserWarning: [W095] Model 'fr_core_news_md' (3.8.0) was trained with spaCy v3.8.0 and may not be 100% compatible with the current version (3.7.5). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)
Building inverted index: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 12048/12048 [00:00<00:00, 24306.78it/s]


In [30]:

def preprocess(chunks):
    fulltext = " ".join(chunks)
    tokens = nlp_fr(fulltext)
    res = []
    for token in tokens:        
        if not token.is_punct and not token.is_space:
            res.append(token.lemma_)
    return " ".join(res)

def postprocess(result):
    doc = docs[docs['cdtn_id'] == result['id']].to_dict(orient='records')[0]    
    return {"chunk":{"metadata":{"url": doc["url"], "id": doc["cdtn_id"]}, "content": " ".join(doc["content_chunked"])}, "score":result["score"]}    

    
def search_bm25(query):
    q_prepro = preprocess([query])
    res = se.search(q_prepro, cutoff=120)
    
    chunks = [postprocess(r) for r in res]
    return chunks

In [31]:
search_bm25(questions.loc[14]['question'])[0]

{'chunk': {'metadata': {'url': 'https://code.travail.gouv.fr/fiche-service-public/duree-du-travail-dun-salarie-a-temps-plein',
   'id': 'abdd8263ba'},
  'content': "Combien d'heures de travail pouvez-vous effectuer par jour, par semaine, par mois et par an ? Est-il possible de travailler plus longtemps que la durée légale le prévoit ? Nous vous expliquons vos droits et vos obligations concernant la durée du temps de travail. Quelle est la durée légale de temps de travail ? La durée légale de temps de travail dépend de votre situation. Cas général La durée légale du travail pour un temps complet est fixée à : 35 heures par semaine 151,67 heures par mois 1 607 heures par an Cependant, des dispositions conventionnelles peuvent prévoir une durée de travail hebdomadaire supérieure ou inférieure à 35 heures. Les heures effectuées au-delà de la durée légale sont considérées comme des heures supplémentaires . Vous êtes cadre dirigeant En tant que cadre dirigeant, vo

In [84]:
results_albert = collections.search(questions.loc[14]['question'], [contributions_col, info_col, fiche_mt_col, fiche_sp_col], 128)
results_albert[0]

{'method': 'semantic',
 'score': 0.69506603,
 'chunk': {'object': 'chunk',
  'id': 1,
  'metadata': {'id': 'b9c2c2f2f1',
   'url': 'https://code.travail.gouv.fr/fiche-ministere-travail/le-travail-du-dimanche',
   'source': 'page_fiche_ministere_travail',
   'title': 'Le travail du dimanche',
   'collection_id': 735,
   'document_id': 746753,
   'document_name': 'Le travail du dimanche.json',
   'document_created_at': 1748005919},
  'content': "Dans les industries ou les entreprises industrielles, une convention ou un accord d'entreprise ou d'établissement ou à défaut, une convention ou un accord de branche étendu peut prévoir la possibilité d'organiser le travail de façon continue pour des raisons économiques et d'attribuer le repos hebdomadaire par roulement. Certains salariés seront donc amenés à travailler le dimanche.  \nÀ défaut de convention ou d'accord collectif de travail étendu ou de convention ou d'accord d'entreprise, une dérogation au repos dominical peut être accordée par 

In [93]:
results_bm25 = search_bm25(questions.loc[14]['question'])
results_bm25[0]

{'chunk': {'metadata': {'url': 'https://code.travail.gouv.fr/fiche-service-public/duree-du-travail-dun-salarie-a-temps-plein',
   'id': 'abdd8263ba'},
  'content': "Combien d'heures de travail pouvez-vous effectuer par jour, par semaine, par mois et par an ? Est-il possible de travailler plus longtemps que la durée légale le prévoit ? Nous vous expliquons vos droits et vos obligations concernant la durée du temps de travail. Quelle est la durée légale de temps de travail ? La durée légale de temps de travail dépend de votre situation. Cas général La durée légale du travail pour un temps complet est fixée à : 35 heures par semaine 151,67 heures par mois 1 607 heures par an Cependant, des dispositions conventionnelles peuvent prévoir une durée de travail hebdomadaire supérieure ou inférieure à 35 heures. Les heures effectuées au-delà de la durée légale sont considérées comme des heures supplémentaires . Vous êtes cadre dirigeant En tant que cadre dirigeant, vo

In [ ]:
chunk['metadata']['url']
    content = chunk['content']
    score = f'{chunk['score']

In [ ]:
run_dict = {
    "q_1": {
        "d_12": 0.9,
        "d_23": 0.8,
        "d_25": 0.7,
        "d_36": 0.6,
        "d_32": 0.5,
        "d_35": 0.4,
    },
    "q_2": {
        "d_12": 0.9,
        "d_11": 0.8,
        "d_25": 0.7,
        "d_36": 0.6,
        "d_22": 0.5,
        "d_35": 0.4,
    },
}


In [102]:
from ranx import fuse, Run

# todo deal with multiple matches
formatted_albert = {c['chunk']["metadata"]["id"]:c["score"] for c in results_albert}
run_albert = Run({"14" : formatted_albert})

In [103]:
formatted_bm25 = {c['chunk']["metadata"]["id"]:c["score"] for c in results_bm25}
run_bm25 = Run({"14" : formatted_bm25})

In [115]:
formatted_albert

{'b9c2c2f2f1': 0.57793665,
 '57a22d7a6c': 0.5952884,
 '72a440e404': 0.5988307,
 'd094601132': 0.5935703,
 '7bd7e2ee64': 0.5889483,
 '1e9dda80aa': 0.57754314,
 'd8131643b9': 0.5735023,
 '30fbfc0fc5': 0.59308183,
 '49353eade2': 0.5797432,
 'be4ca37b56': 0.57327914,
 '617a4e4045': 0.5821527,
 '343af01e28': 0.5867203,
 'fd362898df': 0.61112905,
 '8a1c8b5007': 0.60900664,
 'f6e73ca640': 0.57420266,
 'ef70331d5f': 0.6060802,
 'baf42db024': 0.59876263,
 '394e25d76c': 0.58594126,
 '51afbbbe4e': 0.5985574,
 'e3c27df0bb': 0.58070487,
 '9045a08044': 0.5982444,
 'f81be1df7e': 0.5841069,
 '15fc023268': 0.57593346,
 '6b800529ea': 0.58895075,
 '1f63bfcedf': 0.59257805,
 '195bc00ae8': 0.5913188,
 'ba22bf2ee7': 0.59099066,
 '3a8e55b1dd': 0.5907236,
 '42254a1cfe': 0.59022236,
 'abdd8263ba': 0.58753,
 'f8dfde2939': 0.5873863,
 '2848eed9a6': 0.58613944,
 '8dba466e71': 0.58613944,
 'b340e85721': 0.58517176,
 '5ee8a91607': 0.58401835,
 '6d7a8c9838': 0.58362687,
 '28a508b89a': 0.58255845,
 'ee0a4916e0': 0.58

In [116]:
formatted_bm25

{'abdd8263ba': 35.702667,
 '2c05d436fa': 35.12298,
 'ef70331d5f': 34.964256,
 '51afbbbe4e': 33.836063,
 '21fd152b4b': 32.92693,
 '8328342034': 32.500423,
 'ee0a4916e0': 31.88616,
 '485245824c': 31.876179,
 '1f63bfcedf': 31.832346,
 'ae178d220a': 31.437603,
 'd094ac1f12': 31.194963,
 '6e966d5533': 31.17833,
 '2ff6238f3c': 31.173624,
 '5ee8a91607': 31.071028,
 'e3c27df0bb': 31.051208,
 '1384e73a99': 31.03329,
 '1fd59bd075': 30.998474,
 '3e570b735e': 30.973095,
 '8fb93a6f55': 30.966835,
 'f92a11c873': 30.888237,
 '79448100ac': 30.888237,
 '8d8b398f23': 30.869095,
 'c9fbe137f5': 30.800598,
 '6d7a8c9838': 30.788692,
 '576c7468d4': 30.636189,
 'fa00a23389': 30.608618,
 '617a4e4045': 30.433525,
 '80cb214a6b': 30.339228,
 'ca22e31c7a': 30.080095,
 'e14e55ab9d': 30.063332,
 '8eb9e378fc': 29.946606,
 '7053fb8db0': 29.212759,
 '51a5681cf2': 29.202127,
 'cd50e697bf': 29.145643,
 '4f91a78be7': 28.765682,
 'fa19e95d20': 28.70566,
 'fc90105907': 28.241669,
 '6bddb321c4': 28.112116,
 'b264e870bd': 28.

{'ef70331d5f': 0.031746031746031744,
 '51afbbbe4e': 0.030776515151515152,
 'abdd8263ba': 0.02921395544346364,
 '1f63bfcedf': 0.028577260665441927,
 'ee0a4916e0': 0.026161328190508135,
 '5ee8a91607': 0.0251414204902577,
 'e3c27df0bb': 0.023859649122807018,
 '195bc00ae8': 0.023412698412698413,
 '6d7a8c9838': 0.023399014778325122,
 'baf42db024': 0.022377622377622378,
 '617a4e4045': 0.022363818090954522,
 'd094601132': 0.02062772294834178,
 '30fbfc0fc5': 0.02038327526132404,
 '49353eade2': 0.018963675213675216,
 '3f89f16226': 0.018499573742540493,
 'fd362898df': 0.01639344262295082,
 '2c05d436fa': 0.016129032258064516,
 '8a1c8b5007': 0.016129032258064516,
 '72a440e404': 0.015625,
 '21fd152b4b': 0.015384615384615385,
 '8328342034': 0.015151515151515152,
 '74c36ddb50': 0.015101043748612035,
 '9045a08044': 0.014925373134328358,
 '485245824c': 0.014705882352941176,
 '57a22d7a6c': 0.014705882352941176,
 'ae178d220a': 0.014285714285714285,
 'd094ac1f12': 0.014084507042253521,
 '6e966d5533': 0.01

In [23]:
from ranx import fuse, Run

def row_to_chunk(cdtn_id, score):    
    doc = docs[docs['cdtn_id'] == cdtn_id].to_dict(orient='records')[0]    
    return {"score": score, "chunk":{"metadata":{"url": doc["url"], "id": doc["cdtn_id"]}, "content": " ".join(doc["content_chunked"])}, "n_chunks":len(doc["content_chunked"])}    

def combined_search(query):
    k = "q"
    
    results_albert = collections.search(query, [contributions_col, info_col, fiche_mt_col, fiche_sp_col], 128)
    formatted_albert = {c['chunk']["metadata"]["id"]:c["score"] for c in results_albert}
    run_albert = Run({k : formatted_albert})
    
    results_bm25 = search_bm25(query)
    formatted_bm25 = {c['chunk']["metadata"]["id"]:c["score"] for c in results_bm25}
    run_bm25 = Run({k : formatted_bm25})

    combined_test_run = fuse(
        runs=[run_albert, run_bm25],  
        norm="min-max",       
        method="rrf"        
    )

    return [row_to_chunk(id, score) for id, score in combined_test_run[k].items()]
    

In [3]:
question = "coucou"

In [1]:
from FlagEmbedding import BGEM3FlagModel
model = BGEM3FlagModel('BAAI/bge-m3', use_fp16=True)

Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

In [ ]:
model.encode([question], return_dense=True, return_sparse=True, return_colbert_vecs=True, max_length=1024)

In [5]:
docs['full_content'] = docs['content_chunked'].apply(lambda x: " /n ".join(x))

In [10]:
res = model.encode(docs['full_content'][0:200].tolist(), return_dense=True, return_sparse=False, return_colbert_vecs=False, max_length=1024)

KeyboardInterrupt: 

In [8]:
res['dense_vecs'].shape

(2, 1024)